In [1]:
import numpy as np
import matplotlib.pyplot as plt
import astropy.constants as const
import pandas as pd
import cmcrameri as cmcm
import juanfit

In [3]:
df = pd.read_excel("../save/eis_line_list_final.xlsx",sheet_name="resolve")
df["ion"] = df["ion"].str.strip()
df["charge_stat"] = df["charge_stat"].str.strip()
df["quality"] = df["quality"].str.strip()
df

,ion,charge_stat,Z,A,temp,wvl_fit,wvl_chianti,fwhm_fit,fwhm_err,fwhm_true,fwhm_awsom_fit,fwhm_awsom_err,fwhm_awsom_true,quality,desc
0,Fe,x,9,55.85000,5.9,174.417,174.531,0.079,0.002,0.0380,0.082,0.000030,0.0434,a,NaN
1,Fe,ix,8,55.85000,5.8,176.866,176.945,0.065,0.008,NaN,0.081,0.000800,0.0414,c,NaN
2,Fe,x,9,55.85000,5.9,177.125,177.240,0.081,0.001,0.0407,0.083,0.000200,0.0440,a,NaN
3,Fe,xi,10,55.85000,6.0,182.058,182.167,0.079,0.002,0.0381,0.086,0.000100,0.0503,a,NaN
4,Fe,x,9,55.85000,5.9,182.199,182.307,0.087,0.006,0.0517,0.082,0.000500,0.0423,b,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
80,Mg,vii,6,24.30500,5.8,280.617,280.742,0.078,0.010,0.0362,0.097,0.000008,0.0676,b,NaN
81,Al,ix,8,26.98154,5.9,282.305,282.422,0.093,0.020,0.0629,0.103,0.000080,0.0755,b,NaN
82,Al,ix,8,26.98154,5.9,283.922,284.042,0.091,0.008,0.0594,0.113,0.001000,0.0882,b,NaN
83,Fe,xv,14,55.85000,6.2,284.051,284.163,0.056,0.009,NaN,0.094,0.002000,0.0623,c,sim too hot


In [15]:
def aver_err(data,err,thresh=1.e-5,iter_max=1000):
    sigma_sq = np.var(data)
    n_data = len(data)

    if type(data) is list:
        data = np.array(data)
    if type(err) is list:
        err = np.array(err)

    err_sq_mean = np.mean(np.square(err)) 
    for ii in range(iter_max):
        weight = 1./(sigma_sq + err**2)
        weight = weight/np.sum(weight)
        aver_new = np.sum(weight*data)
        sigma_new_sq = np.max([0.,n_data/(n_data - 1.)*np.sum(weight*np.square(data-aver_new)) - \
                                err_sq_mean])
        
        #print(aver_new,sigma_new_sq)
        if ((sigma_new_sq <= sigma_sq*(1+thresh)) and (sigma_sq <= sigma_new_sq*(1+thresh))):
            break
        else:
            sigma_sq = sigma_new_sq
        
    # if sigma_new_sq == 0:
    #     print("sigma_new is 0.")

    sigma_tot = np.sqrt(1./np.sum(1./(sigma_new_sq + err**2)))
    return aver_new, sigma_tot, sigma_new_sq


def bootstrap(data,err,thresh=1.e-5,step=10000):
    aver_new, sigma_tot, sigma_new_sq = aver_err(data,err,thresh)
    aver_bootstrap = np.zeros(step)

    for ii in range(step):
        data = np.random.normal(aver_new,np.sqrt(sigma_new_sq),len(data))
        epsilon = np.random.normal(0,err)
        data_gen = data + epsilon
        aver_bootstrap[ii], _, _ = aver_err(data_gen,err) 
    
    best_low3sig,best_low1sig,best_est,best_up1sig,best_up1sigbest_up3sig = \
            np.percentile(aver_bootstrap,[0.15,16,50,84,99.85])

    return best_low3sig,best_low1sig,best_est,best_up1sig,best_up1sigbest_up3sig

In [10]:
wvl_fit_sw = np.array([174.417,185.104,186.496,193.405,195.015,197.751,201.947,207.344])

wvl_err_sw = np.array([0.001,0.0004,0.0005,0.0002,0.001,0.0001,0.0008,0.0009])

wvl_chianti_sw = np.array([174.531,185.213,186.598,193.509,195.119,197.854,202.044,207.449])

In [18]:
wvl_fit_lw = np.array([253.837,256.81,258.261,259.39,274.077,275.253])

wvl_err_lw = np.array([0.0009,0.002,0.001,0.005,0.002,0.0003])

wvl_chianti_lw = np.array([253.956,256.919,258.374,259.496,274.180,275.361])


In [16]:
bootstrap(wvl_chianti_sw-wvl_fit_sw,wvl_err_sw)

(0.09952133580763815,
 0.10296605749445459,
 0.1046947328963299,
 0.10646741441597558,
 0.10987208822427301)

In [19]:
bootstrap(wvl_chianti_lw-wvl_fit_lw,wvl_err_lw)

(0.10358995632698034,
 0.10783996275125246,
 0.1101002967785085,
 0.11240048069682579,
 0.11715660100154866)